# 来自Transformers的双向编码器表示（BERT）

We have introduced several word embedding models for natural language understanding. After pretraining, the output can be thought of as a matrix where each row is a vector that represents a word of a predefined vocabulary. In fact, these word embedding models are all context-independent. Let’s begin by illustrating this property.

我们已经介绍了几种用于自然语言理解的词嵌入模型。在预训练之后，输出可以被认为是一个矩阵，其中每一行都是一个表示预定义词表中词的向量。事实上，这些词嵌入模型都是与上下文无关的。让我们先来说明这个性质。

## 从上下文无关到上下文敏感

回想一下 :numref:`sec_word2vec_pretraining`和 :numref:`sec_synonyms`中的实验。例如，word2vec和GloVe都将相同的预训练向量分配给同一个词，而不考虑词的上下文（如果有的话）。形式上，任何词元 $x$ 的上下文无关表示是函数 $f(x)$ ，其仅将 $x$ 作为其输入。考虑到自然语言中丰富的多义现象和复杂的语义，上下文无关表示具有明显的局限性。例如，在“a crane is flying”（一只鹤在飞）和“a crane driver came”（一名吊车司机来了）的上下文中，“crane”一词有完全不同的含义；因此，同一个词可以根据上下文被赋予不同的表示。

这推动了“上下文敏感”词表示的发展，其中词的表征取决于它们的上下文。因此，词元 $x$ 的上下文敏感表示是函数 $f(x, c(x))$，其取决于 $x$ 及其上下文 $c(x)$。流行的上下文敏感表示包括TagLM（language-model-augmented sequence tagger，语言模型增强的序列标记器） :cite:`Peters.Ammar.Bhagavatula.ea.2017`、CoVe（Context Vectors，上下文向量） :cite:`McCann.Bradbury.Xiong.ea.2017`和ELMo（Embeddings from Language Models，来自语言模型的嵌入） :cite:`Peters.Neumann.Iyyer.ea.2018`。

例如，通过将整个序列作为输入，ELMo是为输入序列中的每个单词分配一个表示的函数。具体来说，ELMo将来自预训练的双向长短期记忆网络的所有中间层表示组合为输出表示。然后，ELMo的表示将作为附加特征添加到下游任务的现有监督模型中，例如通过将ELMo的表示和现有模型中词元的原始表示（例如GloVe）连结起来。一方面，在加入ELMo表示后，冻结了预训练的双向LSTM模型中的所有权重。另一方面，现有的监督模型是专门为给定的任务定制的。利用当时不同任务的不同最佳模型，添加ELMo改进了六种自然语言处理任务的技术水平：情感分析、自然语言推断、语义角色标注、共指消解、命名实体识别和问答。

## 从特定于任务到不可知任务

尽管ELMo显著改进了各种自然语言处理任务的解决方案，但每个解决方案仍然依赖于一个特定于任务的架构。然而，为每一个自然语言处理任务设计一个特定的架构实际上并不是一件容易的事。GPT（Generative Pre Training，生成式预训练）模型为上下文的敏感表示设计了通用的任务无关模型 :cite:`Radford.Narasimhan.Salimans.ea.2018`。GPT建立在Transformer解码器的基础上，预训练了一个用于表示文本序列的语言模型。当将GPT应用于下游任务时，语言模型的输出将被送到一个附加的线性输出层，以预测任务的标签。与ELMo冻结预训练模型的参数不同，GPT在下游任务的监督学习过程中对预训练Transformer解码器中的所有参数进行微调。GPT在自然语言推断、问答、句子相似性和分类等12项任务上进行了评估，并在对模型架构进行最小更改的情况下改善了其中9项任务的最新水平。

然而，由于语言模型的自回归特性，GPT只能向前看（从左到右）。在“i went to the bank to deposit cash”（我去银行存现金）和“i went to the bank to sit down”（我去河岸边坐下）的上下文中，由于“bank”对其左边的上下文敏感，GPT将返回“bank”的相同表示，尽管它有不同的含义。

## BERT：把两个最好的结合起来

As we have seen, ELMo encodes context bidirectionally but uses task-specific architectures; while GPT is task-agnostic but encodes context left-to-right.

如我们所见，ELMo 对上下文进行双向编码，但使用特定于任务的架构；而 GPT 是任务无关的，但是从左到右编码上下文。

Combining the best of both worlds, BERT (Bidirectional Encoder Representations from Transformers) encodes context bidirectionally and requires minimal architecture changes for a wide range of natural language processing tasks (Devlin et al., 2018). Using a pretrained Transformer encoder, BERT is able to represent any token based on its bidirectional context.

BERT（来自Transformers的双向编码器表示）融合了两者的优点，双向编码上下文，并且只需对模型架构进行最小的改动，就能适用于广泛的自然语言处理任务（Devlin 等人，2018 年）。通过使用预训练的 Transformer 编码器，BERT 能够基于双向上下文表示任何 token。

During supervised learning of downstream tasks, BERT is similar to GPT in two aspects.
* First, BERT representations will be fed into an added output layer, with minimal changes to the model architecture depending on nature of tasks, such as predicting for every token vs. predicting for the entire sequence.
* Second, all the parameters of the pretrained Transformer encoder are fine-tuned, while the additional output layer will be trained from scratch. Fig. 15.8.1 depicts the differences among ELMo, GPT, and BERT.

在下游任务的监督学习过程中，BERT在两个方面与GPT相似
* 首先，BERT表示将被输入到一个添加的输出层中，根据任务的性质，对模型架构进行最小的改动，例如针对每个标记进行预测与针对整个序列进行预测。
* 其次，对预训练Transformer编码器的所有参数进行微调，而额外的输出层将从头开始训练。 :numref:`fig_elmo-gpt-bert` 描述了ELMo、GPT和BERT之间的差异。

![ELMo、GPT和BERT的比较](../images/elmo-gpt-bert.png)
:label:`fig_elmo-gpt-bert`

BERT further improved the state of the art on eleven natural language processing tasks under broad categories of
* (i) single text classification (e.g., sentiment analysis),
* (ii) text pair classification (e.g., natural language inference),
* (iii) question answering,
* (iv) text tagging (e.g., named entity recognition).

All proposed in 2018, from context-sensitive ELMo to task-agnostic GPT and BERT, conceptually simple yet empirically powerful pretraining of deep representations for natural languages have revolutionized solutions to various natural language processing tasks.

BERT2018年提出进一步改进了11种自然语言处理任务的技术水平，这些任务分为以下几个大类：
* （1）单一文本分类（如情感分析）
* （2）文本对分类（如自然语言推断）
* （3）问答
* （4）文本标记（如命名实体识别）

从上下文敏感的 ELMo 到通用任务的 GPT 和 BERT，这些概念上简单但经验上强大的自然语言深度表示预训练已经彻底改变了各种自然语言处理任务的解决方案。

In the rest of this chapter, we will dive into the pretraining of BERT. When natural language processing applications are explained in Section 16, we will illustrate fine-tuning of BERT for downstream applications.

在本章的其余部分，我们将深入了解BERT的 pretraining 。当在 :numref:`chap_nlp_app`中解释自然语言处理应用时，我们将说明针对下游应用的BERT微调。


In [3]:
import torch
from torch import nn
from d2l import torch as d2l

## 输入表示
:label:`subsec_bert_input_rep`

In natural language processing, some tasks (e.g., sentiment analysis) take single text as input, while in some other tasks (e.g., natural language inference), the input is a pair of text sequences. The BERT input sequence unambiguously represents both single text and text pairs. In the former, the BERT input sequence is the concatenation of the special classification token “\<cls\>”, tokens of a text sequence, and the special separation token “\<sep\>”. In the latter, the BERT input sequence is the concatenation of “\<cls\>”, tokens of the first text sequence, “\<sep\>”, tokens of the second text sequence, and “\<sep\>”. We will consistently distinguish the terminology “BERT input sequence” from other types of “sequences”. For instance, one BERT input sequence may include either one text sequence or two text sequences.

在自然语言处理中，有些任务（如情感分析）以单个文本作为输入，而有些任务（如自然语言推断）以一对文本序列作为输入。BERT输入序列明确地表示单个文本和文本对。当输入为单个文本时，BERT输入序列是特殊类别词元“&lt;cls&gt;”、文本序列的标记、以及特殊分隔词元“&lt;sep&gt;”的连结。当输入为文本对时，BERT输入序列是“&lt;cls&gt;”、第一个文本序列的标记、“&lt;sep&gt;”、第二个文本序列标记、以及“&lt;sep&gt;”的连结。我们将始终如一地将术语“BERT输入序列”与其他类型的“序列”区分开来。例如，一个*BERT输入序列*可以包括一个*文本序列*或两个*文本序列*。

为了区分文本对，根据输入序列学到的片段嵌入 $\mathbf{e}_A$ 和 $\mathbf{e}_B$ 分别被添加到第一序列和第二序列的词元嵌入中。对于单文本输入，仅使用$ \mathbf{e}_A$。

下面的`get_tokens_and_segments`将一个句子或两个句子作为输入，然后返回BERT输入序列的标记及其相应的片段索引。


In [2]:
#@save
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """获取输入序列的词元及其片段索引"""
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 0和1分别标记片段A和B
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments

BERT chooses the Transformer encoder as its bidirectional architecture. Common in the Transformer encoder, positional embeddings are added at every position of the BERT input sequence. However, different from the original Transformer encoder, BERT uses learnable positional embeddings. To sum up, Fig. 15.8.2 shows that the embeddings of the BERT input sequence are the _sum_ of the _token embeddings_ , _segment embeddings_, and _positional embeddings_.

BERT 选择 Transformer 编码器作为其双向架构。在 Transformer 编码器中常见是，位置嵌入被加入到输入序列的每个位置。然而，与原始的 Transformer 编码器不同，BERT使用 *可学习的* 位置嵌入。总之，
:numref:`fig_bert-input`表明BERT输入序列的嵌入是 _词元嵌入_、*片段嵌入* 和 *位置嵌入* 的 *和*。

![BERT输入序列的嵌入是词元嵌入、片段嵌入和位置嵌入的和](../images/bert-input.png)
:label:`fig_bert-input`

下面的`BERTEncoder`类类似于 :numref:`sec_transformer`中实现的`TransformerEncoder`类。与`TransformerEncoder`不同，`BERTEncoder`使用片段嵌入和可学习的位置嵌入。


In [21]:
#@save
class BERTEncoder(nn.Module):
    """BERT encoder."""
    def __init__(self, vocab_size, num_hiddens, ffn_num_hiddens, num_heads,
                 num_blks, dropout, max_len=1000, **kwargs):
        super(BERTEncoder, self).__init__(**kwargs)
        self.token_embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=num_hiddens)
        self.segment_embedding = nn.Embedding(num_embeddings=2, embedding_dim=num_hiddens)
        self.blks = nn.Sequential()
        for i in range(num_blks):
            self.blks.add_module(f"{i}", d2l.TransformerEncoderBlock(
                num_hiddens, ffn_num_hiddens, num_heads, dropout, True))
        # In BERT, positional embeddings are learnable, thus we create a parameter of positional embeddings that are long enough
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, num_hiddens))

    def forward(self, tokens, segments, valid_lens):
        # Shape of `X` remains unchanged in the following code snippet: (batch size, max sequence length, `num_hiddens`)
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        X = X + self.pos_embedding[:, :X.shape[1], :]
        for blk in self.blks:
            X = blk(X, valid_lens)
        return X

Suppose that the vocabulary size is 10000. To demonstrate forward inference of BERTEncoder, let’s create an instance of it and initialize its parameters.

假设词表大小为10000，为了演示 `BERTEncoder` 的前向推断，让我们创建一个实例并初始化它的参数。


In [22]:
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
ffn_num_input, num_blks, dropout = 768, 2, 0.2
encoder = BERTEncoder(vocab_size, num_hiddens, ffn_num_hiddens, num_heads,
                      num_blks, dropout)

We define tokens to be 2 BERT input sequences of length 8, where each token is an index of the vocabulary. The forward inference of BERTEncoder with the input tokens returns the encoded result where each token is represented by a vector whose length is predefined by the hyperparameter num_hiddens. This hyperparameter is usually referred to as the hidden size (number of hidden units) of the Transformer encoder.

我们将 `tokens` 定义为长度为 8 的 2 个 BERT 输入序列，其中每个 `token` 是词表的索引。将输入标记传递给 BERTEncoder 进行前向推理，会返回编码结果，其中每个标记都由一个向量表示，该向量的长度由超参数 `num_hiddens` 预先定义。这个超参数通常被称为 Transformer 编码器的隐藏大小（隐藏单元的数量）。


In [64]:
tokens = torch.randint(0, vocab_size, (2, 8))
print(tokens)
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
print(segments)
encoded_X = encoder(tokens, segments, None)
encoded_X.shape

tensor([[8199, 6420, 1893, 1823, 2040, 5030, 3669, 6290],
        [6414, 4864, 9736, 8647, 4365, 3044, 1228, 2032]])
tensor([[0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 1, 1]])


torch.Size([2, 8, 768])

## 预训练任务
:label:`subsec_bert_pretraining_tasks`

The forward inference of BERTEncoder gives the BERT representation of each token of the input text and the inserted special tokens “\<cls\>” and “\<seq\>”. Next, we will use these representations to compute the loss function for pretraining BERT. The pretraining is composed of the following two tasks: masked language modeling and next sentence prediction.

`BERTEncoder`的前向推断给出了输入文本的每个词元和插入的特殊标记“&lt;cls&gt;”及“&lt;seq&gt;”的BERT表示。接下来，我们将使用这些表示来计算预训练BERT的损失函数。预训练包括以下两个任务：掩蔽语言模型和下一句预测。

### 掩蔽语言模型（Masked Language Modeling）
:label:`subsec_mlm`

As illustrated in Section 9.3, a language model predicts a token using the context on its left.
To encode context bidirectionally for representing each token, BERT randomly masks tokens and uses tokens from the bidirectional context to predict the masked tokens in a self-supervised fashion. This task is referred to as a masked language model.

如 :numref:`sec_language_model`所示，语言模型使用左侧的上下文预测词元。
为了双向编码上下文以表示每个词元，BERT 随机掩蔽 tokens 并使用 `双向上下文` 中的 tokens 以自监督的方式预测掩蔽 tokens。此任务称为 *掩蔽语言模型*。

在这个预训练任务中，将随机选择15%的词元作为预测的掩蔽词元。要预测一个掩蔽词元而不使用标签作弊，一个简单的方法是总是用一个特殊的 “&lt;mask&gt;” 替换输入序列中的词元。然而，这种人工的特殊标记 “&lt;mask&gt;” 不会出现在微调中。为了避免预训练和微调之间的这种不匹配，如果某个标记被选为掩码标记进行预测（例如，在“this movie is great”中，“great”被选为掩码标记进行预测），则在输入中它将被替换为：

* 80%的时候使用一个特殊的“&lt;mask&gt;“词元（例如，“this movie is great”变为“this movie is&lt;mask&gt;”；
* 10%的时间随机替换一个词（例如，“this movie is great”变为“this movie is drink”）；
* 10%的情况保持标签标记不变（例如，“this movie is great”变为“this movie is great”）。

Note that for 10% of 15% time a random token is inserted. This occasional noise encourages BERT to be less biased towards the masked token (especially when the label token remains unchanged) in its bidirectional context encoding.

请注意，在 15% 的时间里会有 10% 的概率插入一个随机标记。这种偶尔出现的干扰有助于 BERT 在双向上下文编码过程中减少对掩码标记的偏向（尤其是在标签标记保持不变的情况下）。

We implement the following MaskLM class to `predict masked tokens` in the masked language model task of BERT pretraining. The prediction uses a one-hidden-layer MLP (self.mlp). In forward inference, it takes two inputs: the encoded result of BERTEncoder and the token positions for prediction. The output is the prediction results at these positions.

我们实现了下面的 `MaskLM` 类来 `预测` BERT预训练的掩蔽语言模型任务中的 `掩蔽标记`。预测使用单隐藏层的多层感知机（`self.mlp`）。在前向推断中，它需要两个输入：`BERTEncoder` 的 `编码结果` 和  `用于预测的词元位置`。输出是这些位置的预测结果。


In [49]:
#@save
class MaskLM(nn.Module):
    """BERT的掩蔽语言模型任务"""
    def __init__(self, vocab_size, num_hiddens, num_inputs=768, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        self.mlp = nn.Sequential(nn.Linear(num_inputs, num_hiddens),
                                 nn.ReLU(),
                                 nn.LayerNorm(num_hiddens),
                                 nn.Linear(num_hiddens, vocab_size))

    def forward(self, X, pred_positions):
        num_pred_positions = pred_positions.shape[1] # 3
        pred_positions = pred_positions.reshape(-1)
        batch_size = X.shape[0] # 2
        batch_idx = torch.arange(0, batch_size) # [0,1]
        # Suppose that `batch_size` = 2, `num_pred_positions` = 3, then
        # `batch_idx` is `torch.tensor([0, 0, 0, 1, 1, 1])`
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions)
        print(f"\n批次索引 batch_idx:\n{batch_idx}")
        print(f"\n批次中的位置索引 pred_positions:\n{pred_positions}")
        print(f"\nX:\n{X}")
        masked_X = X[batch_idx, pred_positions]
        print(f"\nmasked_X:\n{masked_X}")
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))
        mlm_Y_hat = self.mlp(masked_X)
        return mlm_Y_hat

为了演示`MaskLM`的前向推断，我们创建了其实例`mlm`并对其进行了初始化。回想一下，来自`BERTEncoder`的正向推断`encoded_X`表示2个BERT输入序列。我们将`mlm_positions`定义为在`encoded_X`的任一输入序列中预测的3个指示。`mlm`的前向推断返回`encoded_X`的所有掩蔽位置`mlm_positions`处的预测结果`mlm_Y_hat`。对于每个预测，结果的大小等于词表的大小。


In [50]:
mlm = MaskLM(vocab_size, num_hiddens)
# 在两个 batch 中，从每个 batch 的 8 个 tokens 中选了 3 个 tokens 作为 masked X，传入到 mlm 中进行预测
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
mlm_Y_hat = mlm(encoded_X, mlm_positions)
mlm_Y_hat.shape


批次索引 batch_idx:
tensor([0, 0, 0, 1, 1, 1])

批次中的位置索引 pred_positions:
tensor([1, 5, 2, 6, 1, 5])

X:
tensor([[[-0.1335,  1.5813,  0.8589,  ...,  1.3260,  0.7286, -0.7546],
         [-1.1188, -0.0318, -0.2060,  ...,  2.0960, -1.0534,  0.1013],
         [-0.7444,  1.2326,  0.3800,  ...,  0.9313, -0.0984,  0.8084],
         ...,
         [ 0.4786, -0.8220, -0.0899,  ..., -0.7513, -1.4279, -0.9471],
         [ 1.6791, -0.1813, -0.5031,  ..., -1.3655,  0.5331, -1.6252],
         [ 0.6507, -0.2169, -1.7172,  ..., -0.5537,  0.4931, -0.2818]],

        [[-0.2761,  1.6569,  0.4204,  ...,  3.2492,  1.3905, -2.1551],
         [-0.7220, -0.0359,  0.1230,  ...,  0.5475, -0.4374,  1.3512],
         [-1.7443,  1.0358, -0.3974,  ...,  0.1500,  0.6497, -0.5879],
         ...,
         [-0.1530, -1.2651, -1.4331,  ...,  0.7563, -0.2038, -2.0108],
         [ 1.5008, -0.9628,  1.2556,  ..., -0.6753, -0.8390, -1.8912],
         [ 0.5561,  0.7678,  0.3997,  ...,  0.9833,  0.1520, -0.3079]]],
       grad_fn=

torch.Size([2, 3, 10000])

With the ground truth labels mlm_Y of the predicted tokens mlm_Y_hat under masks, we can calculate the cross-entropy loss of the masked language model task in BERT pretraining.

通过掩码下的预测词元 `mlm_Y_hat` 的真实标签 `mlm_Y`，我们可以计算在BERT预训练中的遮蔽语言模型任务的交叉熵损失。


In [51]:
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
loss = nn.CrossEntropyLoss(reduction='none')
mlm_l = loss(input=mlm_Y_hat.reshape((-1, vocab_size)), target=mlm_Y.reshape(-1))
mlm_l.shape, mlm_l

(torch.Size([6]),
 tensor([9.6163, 8.9131, 8.6389, 8.0589, 9.1005, 8.8871],
        grad_fn=<NllLossBackward0>))

### 下一句预测（Next Sentence Prediction）
:label:`subsec_nsp`

Although masked language modeling is able to encode bidirectional context for representing words, it does not explicitly model the logical relationship between text pairs. To help understand the relationship between two text sequences, BERT considers a binary classification task, next sentence prediction, in its pretraining. When generating sentence pairs for pretraining, for half of the time they are indeed consecutive sentences with the label “True”; while for the other half of the time the second sentence is randomly sampled from the corpus with the label “False”.

尽管掩蔽语言建模能够编码双向上下文来表示单词，但它不能显式地建模文本对之间的逻辑关系。为了帮助理解两个文本序列之间的关系，BERT在预训练中考虑了一个二元分类任务—— *下一句预测* 。在为预训练生成句子对时，有一半的时间它们确实是标签为“真”的连续句子；在另一半的时间里，第二个句子是从语料库中随机抽取的，标记为“假”。

The following NextSentencePred class uses a one-hidden-layer MLP to predict whether the second sentence is the next sentence of the first in the BERT input sequence. Due to self-attention in the Transformer encoder, the BERT representation of the special token “\<cls\>” encodes both the two sentences from the input. Hence, the output layer (self.output) of the MLP classifier takes X as input, where X is the output of the MLP hidden layer whose input is the encoded “\<cls\>” token.

下面的`NextSentencePred`类使用 单隐藏层 的多层感知机来预测第二个句子是否是BERT输入序列中第一个句子的下一个句子。由于Transformer编码器中的自注意力，特殊词元“&lt;cls&gt;”的BERT表示已经对输入的两个句子进行了编码。因此，多层感知机分类器 的输出层（`self.output`）以 `X` 作为输入，其中 `X` 是 多层感知机隐藏层 的输出，而 MLP隐藏层 的 输入 是编码后的“&lt;cls&gt;”词元。


In [63]:
#@save
class NextSentencePred(nn.Module):
    """The next sentence prediction task of BERT."""
    def __init__(self, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        self.output = nn.LazyLinear(2)

    def forward(self, X):
        # `X` shape: (batch size, `num_hiddens`)
        return self.output(X)

我们可以看到，`NextSentencePred`实例的前向推断返回每个BERT输入序列的二分类预测。


In [65]:
 # encoded_X = torch.Size([2, 8, 768])
encoded_X = torch.flatten(encoded_X, start_dim=1)
# input_shape for NSP: (batch size, `num_hiddens`) encoded_X = torch.Size([2, 6144])
nsp = NextSentencePred()
nsp_Y_hat = nsp(encoded_X)
nsp_Y_hat.shape

torch.Size([2, 2])

还可以计算两个二元分类的交叉熵损失。


In [66]:
nsp_y = torch.tensor([0, 1])
nsp_l = loss(nsp_Y_hat, nsp_y)
nsp_l.shape

torch.Size([2])

It is noteworthy that all the labels in both the aforementioned pretraining tasks can be trivially obtained from the pretraining corpus without manual labeling effort. The original BERT has been pretrained on the concatenation of BookCorpus (Zhu et al., 2015) and English Wikipedia. These two text corpora are huge: they have 800 million words and 2.5 billion words, respectively.

值得注意的是，上述两个预训练任务中的所有标签都可以从预训练语料库中获得，而无需人工标注。原始的BERT已经在图书语料库 :cite:`Zhu.Kiros.Zemel.ea.2015`和英文维基百科的连接上进行了预训练。这两个文本语料库非常庞大：它们分别有8亿个单词和25亿个单词。

## 整合代码

When pretraining BERT, the final loss function is a linear combination of both the loss functions for masked language modeling and next sentence prediction. Now we can define the BERTModel class by instantiating the three classes BERTEncoder, MaskLM, and NextSentencePred. The forward inference returns the encoded BERT representations encoded_X, predictions of masked language modeling mlm_Y_hat, and next sentence predictions nsp_Y_hat.

在预训练BERT时，最终的损失函数是掩蔽语言模型损失函数和下一句预测损失函数的线性组合。现在我们可以通过实例化三个类`BERTEncoder`、`MaskLM`和`NextSentencePred`来定义`BERTModel`类。前向推断返回编码后的BERT表示`encoded_X`、掩蔽语言模型预测`mlm_Y_hat`和下一句预测`nsp_Y_hat`。


In [12]:
#@save
class BERTModel(nn.Module):
    """The BERT model."""
    def __init__(self, vocab_size, num_hiddens, ffn_num_hiddens,
                 num_heads, num_blks, dropout, max_len=1000):
        super(BERTModel, self).__init__()
        self.encoder = BERTEncoder(vocab_size, num_hiddens, ffn_num_hiddens,
                                   num_heads, num_blks, dropout,
                                   max_len=max_len)
        # 将编码器输出的向量转换为适合分类任务的表示形式
        self.hidden = nn.Sequential(nn.LazyLinear(num_hiddens),
                                    nn.Tanh() # 激活函数，将线性变换后的值压缩到[-1, 1]
                                    )
        self.mlm = MaskLM(vocab_size, num_hiddens)
        self.nsp = NextSentencePred()

    def forward(self, tokens, segments, valid_lens=None, pred_positions=None):
        encoded_X = self.encoder(tokens, segments, valid_lens)
        if pred_positions is not None: # pre-train 掩码模型
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
        # The hidden layer of the MLP classifier for next sentence prediction.
        # 0 is the index of the '<cls>' token， 只对<cls>的位置采用二分类预测。
        nsp_Y_hat = self.nsp(self.hidden(encoded_X[:, 0, :]))
        return encoded_X, mlm_Y_hat, nsp_Y_hat

## 小结

* word2vec和GloVe等词嵌入模型与上下文无关。它们将相同的预训练向量赋给同一个词，而不考虑词的上下文（如果有的话）。它们很难处理好自然语言中的一词多义或复杂语义。
* 对于上下文敏感的词表示，如ELMo和GPT，词的表示依赖于它们的上下文。
* ELMo对上下文进行双向编码，但使用特定于任务的架构（然而，为每个自然语言处理任务设计一个特定的体系架构实际上并不容易）；而GPT是任务无关的，但是从左到右编码上下文。
* BERT结合了这两个方面的优点：它对上下文进行双向编码，并且需要对大量自然语言处理任务进行最小的架构更改。
* BERT输入序列的嵌入是词元嵌入、片段嵌入和位置嵌入的和。
* 预训练包括两个任务：掩蔽语言模型和下一句预测。前者能够编码双向上下文来表示单词，而后者则显式地建模文本对之间的逻辑关系。

               <cls>
encoder -> mlm ----> hidden -> nsp